In [82]:
import pandas as pd
import json
import requests
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)

# Cuomo Campaign Contributions from Board of Elections

I read the CSV, then I take the dollar signs out of the 'Amount' column, ...and convert it to float so I can do calculations.

I convert the 'Date' column to Pandas datetime, so I can sort and filter by date.

In [83]:
df = pd.read_csv("Campaign_Contributions.csv")
df['Amount'] = df['Amount'].replace( '[\$,)]','', regex=True ).astype(float)

df['Date'] = pd.to_datetime(df['Date'])
df['Donation_Date']=df['Date']
df.set_index('Date', inplace=True)
df.sort_index(axis=0, inplace=True)

In [84]:
#This is the date range that gets closest to the "31" number that everyone (including the NYT) is citing
#It's just the 4 years prior to November 2018, so that makes sense.
df['2014-11-01':]['Amount'].sum()

30078564.0

In [85]:
#EDIT: John and Peter want to look at everything from 2010 onward.
df_warchest = df['2010-01-01':]

In [86]:
import difflib

In [87]:
df_warchest[df_warchest['Donor'].str.contains('', na=False)]

,Donor,Address,Filer ID,Check No.,Amount,Recipient,Transaction Code,Contribution Code,Contribution Type,Fiscal Report,Donation_Date
Date,,,,,,,,,,,
2010-01-01,CAESAR ALARCON,"19B E. MAIN STREET INDIAN LAKE, NY, 12842",A31966,CC,25.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/Individual and Partnerships,Individual,NaN,Periodic Jan.,2010-01-01
2010-01-02,JP MORGAN CHASE BANK,"450 THIRD AVENUE NEW YORK, NY, 10016",A31966,NaN,4.0,ANDREW CUOMO 2018 INC.,Other Receipts,NaN,NaN,Periodic Jan.,2010-01-02
2010-01-04,MARC DAVIS,"35 AUDREY AVENUE NEEDHAM, MA, 02492",A31966,1216,500.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/Individual and Partnerships,Individual,NaN,Periodic Jan.,2010-01-04
2010-01-04,CSX TRANSPORTATION ACCOUNTS PAYABLE S/C J682,"P.O. BOX 44057 JACKSONVILLE, FL, 32231",A31966,5034799,1000.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/Corporate,NaN,NaN,Periodic Jan.,2010-01-04
2010-01-04,NEW YORK STATE ARCHITECTS PAC,"52 SOUTH PEARL STREET ALBANY, NY, 12207",A31966,0761,2000.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/All Other,NaN,NaN,Periodic Jan.,2010-01-04
2010-01-04,WELLS FARGO ADVISORS,"51 JFK PARKWAY, 4TH FLOOR SHORT HILLS, NJ, 07078",A31966,NaN,314.0,ANDREW CUOMO 2018 INC.,Other Receipts,NaN,NaN,Periodic Jan.,2010-01-04
2010-01-04,MICHAEL FALCONE,"1111 LAC DE VILLE BLVD ROCHESTER, NY, 14618",A31966,CC,25.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/Individual and Partnerships,Individual,NaN,Periodic Jan.,2010-01-04
2010-01-04,BENJAMIN JAMES,"102 EAST 4TH STREET NEW YORK, NY, 10003",A31966,CC,25.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/Individual and Partnerships,Individual,NaN,Periodic Jan.,2010-01-04
2010-01-04,CONIFER REALTY LLC,"183 EAST MAIN STREET ROCHESTER, NY, 14604",A31966,282793,500.0,ANDREW CUOMO 2018 INC.,Monetary Contributions/All Other,NaN,NaN,Periodic Jan.,2010-01-04


## We know that LLC's give a lot in political campaigns, and they give repeatedly.

They may even be top donors, if they are categorized individually.

In [156]:
#Who are Cuomo's top individual donors, by sum dollar amounts they have given?
#But these are not the REAL top donors, since entities use LLCs and other mechanisms to obscure repeated donations.
df_top = df_warchest.groupby(['Donor','Address'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else list(x)).sort_values(by='Amount', ascending=False)
df_top = df_top.head(50)
_, count = df_top[df_top['Donor'].str.contains("LLC")].shape
print(count,"of Cuomo's top 50 individual donors by total amount given are LLC's")

11 of Cuomo's top 50 individual donors by total amount given are LLC's


## I'm going to cross-reference the list of Cuomo donor LLCs with a nyc-db to see if there are any matches.

In [183]:
llc_list = df_warchest[df_warchest['Donor'].str.contains("LLC",na=False)]

#Here I checked the LLC names against the LLCs listed in DOBJOBS

llc_list.columns = [c.lower() for c in llc_list.columns] #postgres doesn't like capitals or spaces

from sqlalchemy import create_engine
engine = create_engine('postgresql://GeorgiaKromrei@localhost:5432/housing_data_analysis')

llc_list.to_sql("cuomo_llc_list", engine, if_exists="replace")
print("I'm loading", llc_list['donor'].nunique(), "unique LLC names into nyc-db for cross-referencing.")

I'm loading 1254 unique LLC names into nyc-db for cross-referencing.


## So I can answer the question: how many landlords used their LLC's for managing buildings, and also for donating to Cuomo?

I start by running the cuomo_llc_list.csv file through nyc-db's dobjobs table, to find exact matches-- this will be a "minimum" number. I find 132 unique LLCs that are both Cuomo donors and landlord LLCs.

In [184]:
#These are the LLC's I get by matching directly in SQL.

df_llc=pd.read_csv('cuomo_llc_list.csv')
df_llc['ownersphone']=df_llc['ownersphone'].astype(str)
df_llc['ownersbusinessname'].nunique()

132

## But for a more exhaustive list, I'm going to need a list of all the LLC's in New York City.

That looks like about 175,000 unique LLCs. I'm going to match them up to the Cuomo LLC's.
I run the following query on my local copy of nyc-db to get all of the LLC's from DOBJOBS in New York:

```CREATE TABLE nyc_llc AS SELECT ownersbusinessname, bbl, address FROM dobjobs WHERE ownersbusinessname LIKE('%LLC%') GROUP BY ownersbusinessname, bbl, address                                                                 ;
SELECT 175564
```

In [185]:
#If I use Python, I get a few more

import numpy as np
nyc_llc = pd.read_csv('nyc_llc.csv')
nyc_llc['businessname_nollc'] = nyc_llc['ownersbusinessname'].str.upper().str.replace(r',? LLC.*','')
llc_list['donorname_nollc'] = llc_list['donor'].str.replace(r',? LLC.*','')

unique = llc_list[llc_list['donorname_nollc'].isin(nyc_llc['businessname_nollc'])]['donor'].nunique()

print("There are", unique, "unique LLC's in NYC-DB who have given to Cuomo's campaign.")

There are 168 unique LLC's in NYC-DB who have given to Cuomo's campaign.


## I want more granular information on each LLC, how much they've given to Cuomo and when. 

So I won't just check to see if they match to some LLC in Cuomo's donor list: but confirm exactly which one(s) they match.

In [186]:
llc_list = llc_list.merge(nyc_llc, right_on='businessname_nollc', left_on='donorname_nollc', how='left')
llc_list.drop_duplicates('check no.', keep='first', inplace=True)
dumb_landlords = llc_list.groupby(['ownername','address_x'], as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ', '.join(x))

In [187]:
dumb_landlords.rename(columns={'ownername': 'dobjobs_ownername',
                                              'address_x': 'cuomo_donor_address',
                                              'address_y': 'dobjobs_address',
                                              'amount': 'donation_total',
                                              'ownersbusinessname': 'dobjobs_ownersbusinessname',
                                              'ownersphone': 'dobjobs_ownersphone'}, inplace=True)

In [188]:
dumb_landlords.to_csv("dumbs.csv")

# Next step: connect entities to industry.

## Plan:

Working with this data may be more like a war of attrition, with various passes matching the addresses or donor names to companies, 1-to-1.

Passes:

1. NYC-DB -- since I know real estate is such a significant portion of investors in Cuomo, start with NYCDB.
2. Officers -- Find those donor names matching officer names registered with OpenCorporates.
    a. What about common names -- should I limit jurisdiction to New York?
3. Registered addresses -- Find those addresses that match with registered addresses on OpenCorporates

## I'd like to find the rest of the LLC's: the ones that weren't in nyc-db. I turn to OpenCorporates' API.

In [194]:
llc_addresses = llc_list[llc_list['ownersbusinessname'].isnull()]['address_x'].unique()

In [196]:
def get_company_names():
    for donor_address in llc_addresses:
        #If the contribution code is 'Individual', check for officer name!
        address = "&registered_address=" + str(donor_address.replace(",","+").replace(" ","+"))
        base_url= "https://api.opencorporates.com/v0.4/companies/search?q="
        api_token= "&api_token=BSSwhXpP7xJ5couOddoY"
        response = requests.get(base_url+address+api_token)
        data = json.loads(response.content)
        if len(data['results']['companies'])==1:
            print("*******************************************************************")
            print("Original LLC Donor Address: ", donor_address)
            count = 1
            for company in data['results']['companies']:
                if not company['company']['inactive']:
                    print("Matched Registered Address", count)
                    print("Company name:",company['company']['name']," Company type:",company['company']['company_type'])
                    count+=1
            print("\n\n")


In [ ]:
get_company_names()

*******************************************************************
Original LLC Donor Address:  103 WEST 55TH STREET NEW YORK, NY, 10019



*******************************************************************
Original LLC Donor Address:  STATION PLAZA WOODMERE, NY, 11598



*******************************************************************
Original LLC Donor Address:  15-17 EAST 16TH STREET NEW YORK, NY, 10003
Matched Registered Address 1
Company name: RC DOLNER, INC.  Company type: Stock



*******************************************************************
Original LLC Donor Address:  1300 PENNSYLVANIA AVENUE NW, SUITE 700 WASHINGTON, DC, 20004
Matched Registered Address 1
Company name: POTOMAC LAW GROUP, LLC  Company type: Foreign Limited Liability Company



*******************************************************************
Original LLC Donor Address:  30 NORTH LA SALLE CHICAGO, IL, 60602
Matched Registered Address 1
Company name: INTERNATIONAL FELLOWSHIP OF CHRISTIANS & JEWS, IN

# I need just the unique donor names, so I can use Orbis' batch search tool.

So I take out all of the duplicate Donors, keeping the first one. It only throws away exact matches.

In [20]:
df_warchest[~df_warchest.Donor.duplicated(keep='first')]['Donor'].to_frame().to_csv('unique_donors.csv', header=False, index=False)

## But for a more exhaustive list, I'm going to need a list of all the LLC's in New York City.

That looks like about 175,000 unique LLCs. I'm going to match them up to the Cuomo LLC's.
I run the following query on my local copy of nyc-db:

```CREATE TABLE nyc_llc AS SELECT ownersbusinessname, bbl, address FROM dobjobs WHERE ownersbusinessname LIKE('%LLC%') GROUP BY ownersbusinessname, bbl, address                                                                 ;
SELECT 175564
```

In [ ]:
import numpy as np
nyc_llc = pd.read_csv('nyc_llc.csv')
nyc_llc['ownersbusinessname'] = nyc_llc['ownersbusinessname'].str.upper().str.replace(r',? LLC.*','')
llc_list['donor'] = llc_list['donor'].str.replace(r',? LLC','')

unique = llc_list[llc_list['donor'].isin(nyc_llc['ownersbusinessname'])]['donor'].nunique()

print("There are", unique, "unique LLC's in NYC-DB who have given to Cuomo's campaign.")

## I want more granular information on each LLC, how much they've given to Cuomo and when. 

So I won't just check to see if they match to some LLC in Cuomo's donor list: but confirm exactly which one(s) they match.

In [ ]:
llc_list = llc_list.merge(nyc_llc, right_on='ownersbusinessname', left_on='donor', how='left')

In [ ]:
llc_list.drop_duplicates('check no.', keep='first', inplace=True)
llc_list[llc_list['ownername'].str.contains("ESPOSITO", na=False)]
llc_list.groupby('ownername').agg(lambda x : x.sum() if x.dtype=='float64' else ', '.join(x))

## Some summary statistics and information on Cuomo's LLC's.

- What percent of donations count are they?
- What percent of the total donations dollar amount are they?
- Do they give more or less than Cuomo's average donor?

In [ ]:
rows, _ = df_warchest[df_warchest['Donor'].str.contains("LLC",na=False)].shape
total_rows, _ = df_warchest.shape
print(rows, "or", round(rows/total_rows * 100, 2), "% of donations are from LLCs.")
df_llc = df_warchest[df_warchest['Donor'].str.contains("LLC",na=False)]
df_not_llc = df_warchest[~df_warchest['Donor'].str.contains("LLC",na=False)]
print("The total given by LLCs is:", '${:,.2f}'.format(df_llc['Amount'].sum()))
print("Which is", round(df_llc['Amount'].sum()/df_warchest['Amount'].sum()*100, 2), "% of the total dollar amount.")
unique, _ = df_llc.groupby("Address").first().shape
print("There are", unique, "unique LLCs in the past 4 years of campaign contributions.")
avg_donation = df_not_llc['Amount'].mean()
llc_av_donation = df_llc['Amount'].mean()
print("Cuomo's donors give an average of", '${:,.2f}'.format(avg_donation), "while LLCs give an average of", '${:,.2f}'.format(llc_av_donation))

In [ ]:
company_names = {}
df_llc.groupby("Address").first().reset_index().apply(get_company_names, axis=1)

# Once I have all of the company information from Orbis, I need to read in each of the files and concatenate them into a Pandas dataframe

Glob does this nicely, basically you can do unix-style filename syntax so something like \*.txt grabs all of the files in the folder that have the file extension .txt.

In [ ]:
import glob
allFiles = glob.glob("*.txt")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, encoding='utf-16')
    list_.append(df)
frame = pd.concat(list_)

## Unfortunately Orbis puts repeated information (like telephone numbers, etc.) on different rows, which is not what I want.

I want them in a list, all in one cell, with one row per company.
So I group by 'Mark', since there is only one for each company. Then I take all of the different DM Fulll names and put them in a list, reset the index, and set 'Mark' as the index.
Finally, I join the lists of DMs back onto the original dataset.

In [ ]:
frame['Mark']=frame['Mark'].fillna(method='ffill')
frame['DM Full name']=frame['DM Full name'].astype('str')
df_DMs = frame.groupby('Mark')['DM Full name'].apply(', '.join).reset_index().set_index('Mark')
df_DMs.columns = df_DMs.columns.get_level_values(0)
df_DMs = df_DMs.reset_index()
frame = frame[pd.notnull(frame['Company name'])]
frame= frame.reset_index()
frame = frame.join(df_DMs,rsuffix="KEEP")

In [ ]:
naics_df = pd.read_csv('2017_NAICS_Descriptions.csv')
df = pd.merge(frame, naics_df, left_on='NAICS 2017 Core code (4 digits)',right_on='Code',how='left')

In [ ]:
df_warchest['Donor']=df_warchest['Donor'].astype(str)
df['Company name']=df['Company name'].astype(str)
df = df.sort_values(by='Company name')
df_warchest = df_warchest.sort_values(by='Donor')

In [ ]:
#BE CAREFUL to only run this once
import jellyfish

def get_closest_match(x, list_strings):
    best_match = None
    highest_jw = 0
    for current_string in list_strings:
        current_score = jellyfish.jaro_winkler(x, current_string)

        if(current_score > highest_jw):
            highest_jw = current_score
            best_match = current_string
    return best_match

df['Company name'] = df['Company name'].map(lambda x: get_closest_match(x, df_warchest['Donor']))

In [ ]:
df

In [ ]:
merged_donors_companies= df_warchest.merge(df, how='left', right_on='Company name', left_on='Donor')

llc_networks = merged_donors_companies[merged_donors_companies['Donor'].str.contains('LLC')].groupby(['Telephone number'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else set(x))

In [ ]:
df = merged_donors_companies.groupby(['Telephone number'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else set(x))

In [ ]:
df[df['Telephone number']!= 'Credit needed'].sort_values(by='Amount',ascending=False)

In [ ]:
df['Registered Address']=df['Street, no., building etc, line 1'].map(str) + df['Street, no., building etc, line 2'].map(str)
df.columns

In [ ]:
df = df.groupby(['Registered Address'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else set(x))

In [ ]:
df.sort_values(by='Amount',ascending=False)
#df.columns

In [ ]:
llc_networks.sort_values(by='Amount',ascending=False)

In [ ]:
#Scrape PAC information from Open Secrets

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import re

In [ ]:
df_warchest['PAC Search Name'] = df_warchest['Donor']
replace = ['INC.', '\sPOLITICAL ACTION COMMITTEE$', '\sCORP', '\s&\sCO.', '\s$', '\sASSOCIATION','NY-PAC$','PAC$','PAC,$']

pac_df = df_warchest[(df_warchest['Donor'].str.contains('PAC')) | (df_warchest['Donor'].str.contains('POLITICAL ACTION COMMITTEE'))]
pac_df['PAC Search Name'] = pac_df['PAC Search Name'].replace(to_replace=replace,value="",regex=True )
pac_uniques = pac_df.drop_duplicates('PAC Search Name',keep='first')
#driver=webdriver.Chrome()
#pac_uniques.apply(get_pac_industry_info, axis=1)
pac_uniques.reset_index()

In [ ]:
pac_list=[]
def get_pac_industry_info(row):
    driver.get('https://www.opensecrets.org/pacs/search.php')
    driver.find_element_by_class_name('textfield').send_keys(row['PAC Search Name'])
    driver.find_element_by_class_name('btn').click()
    try:
        driver.find_elements_by_tag_name('td')[0].find_element_by_tag_name('a').click()
        
    except:
        try:
            driver.find_element_by_tag_name('td').click()
        except:
            pass
    pac_dict = {}
    try:    
        left_column = driver.find_element_by_id('profileLeftColumn')
        column = left_column.text.split('\n')
        pac_list.append(column)
    except:
        column = "Not found"
        row['PAC Info'] = column

In [ ]:
count=0
for pac in pac_list:
    if pac=='Not found':
        pass
    else:
        count+=1
print(count)

In [ ]:
['href']